In [3]:
import numpy as np
import cv2
from moviepy.editor import *
import math


# 高斯模糊 封装

def blur(image,kernel=(5,5)):
    blur_image = cv2.GaussianBlur(image,kernel,0)
    return blur_image

# Edge Detection 
#Canny or Sobel Algo

def canny(image):
    canny = cv2.Canny(image,250,300)
    return canny


# range of interested 

def roi(image,roi_range):
    
    #define a whole black area with the same size of output goal image 也可以理解为掩膜
    mask = np.zeros_like(image)
    imshape = image.shape
    
    #based on the channel of input image
    if len(image.shape) > 2:
        channel_count = image.shape[2] # denpends on image 3.4 都有可能
        cv2.fillPoly(mask,roi_range,(255,)*channel_count)
    else:
        cv2.fillPoly(mask,roi_range,255)
    
    # '与' 操作 ，combine the ROI 
    img_masked = cv2.bitwise_and(image,mask)
    return img_masked


# 霍夫变换  

def hough_line(image,rho=1,theta=np.pi/180,threshold=15,min_line_len=25,max_line_gap=20):
    
    #霍夫检测线段， 线段的两个端点 坐标存储爱在 lines 中 
    lines = cv2.HoughLinesP(image,rho,theta,threshold,np.array([]),min_line_len,max_line_gap)
    return lines


#  出 检测线 处理+画线 

def process_line(image,lines):
    positive_slop_points = []
    negative_slop_points = []
    positive_slop_intercept = [] #左边线点构成直线的斜率和截距
    negative_slop_intercept = [] #右边。。。
    line_img = np.zeros((image.shape[0],image.shape[1],3),dtype=np.uint8)
    if lines is None:
        return line_img
    for line in lines:
        for x1,y1,x2,y2 in line:
            slop = np.float((y2-y1))/np.float((x2-x1))
            if slop > 0:
                positive_slop_points.append([x1,y1])
                positive_slop_points.append([x2,y2])
                positive_slop_intercept.append([slop,y1-slop*x1])
            elif slop < 0 :
                negative_slop_points.append([x1,y1])
                negative_slop_points.append([x2,y2])
                negative_slop_intercept.append([slop,y1-slop*x1])
    if (len(negative_slop_points) == 0 or len(negative_slop_points) == 0):
        return line_img
    positive_slop,positive_intercept = filter_line(positive_slop_intercept)
    negative_slop,negative_intercept = filter_line(negative_slop_intercept)
    ymin = 325
    ymax = image.shape[0]

    draw_line(line_img,positive_slop,positive_intercept,ymin,ymax)
    draw_line(line_img,negative_slop,negative_intercept,ymin,ymax)
    return line_img


 
def draw_line(image,slop,intercept,ymin,ymax):
    xmin = int((ymin - intercept)/slop)
    xmax = int((ymax-intercept)/slop)
    cv2.line(image,(xmin,ymin),(xmax,ymax),[255,0,0],5)



def filter_line(slop_intercept):
    legal_slop=[]
    legal_intercept=[]
    slopes=[pair[0] for pair in slop_intercept]
    slop_mean = np.mean(slopes)  # 平均斜率 
    slop_std = np.std(slopes) # 斜率标准差
    for pair in slop_intercept:
        if pair[0] - slop_mean < 3 * slop_std:
            legal_slop.append(pair[0])
            legal_intercept.append(pair[1])
    if not legal_slop:
        legal_slop = slopes
        legal_intercept = [pair[1] for pair in slop_intercept]
    slop = np.mean(legal_slop)
    intercept = np.mean(legal_intercept)
    return slop,intercept


def process_picture(image):
    # 1
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) 
    # 2
    blur_image = blur(gray)
    # 3
    canny_image = canny(blur_image)
    imshape = canny_image.shape
    # 4
    roi_range = np.array([[(0,imshape[0]),(465, 320), (475, 320),(imshape[1],imshape[0])]],dtype = np.int32)
    img_masked = roi(canny_image,roi_range)
    # 5
    lines = hough_line(img_masked)
    # 6
    line_image = process_line(img_masked,lines)
    # 7
    res_image = cv2.addWeighted(image,0.7,line_image,1,0)
    return res_image

def process_video(path):
    video = VideoFileClip(path)
    video_after_process = video.fl_image(process_picture)
    video_after_process.write_videofile("./lane_detection.mp4",audio=False)

if __name__ == '__main__':
    process_video('test_videos/solidWhiteRight.mp4')


[MoviePy] >>>> Building video ./lane_detection.mp4
[MoviePy] Writing video ./lane_detection.mp4


100%|█████████▉| 221/222 [00:01<00:00, 160.92it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./lane_detection.mp4 

